In [86]:
import numpy as np 
import pandas as pd 
import yfinance as yf
import quantstats as qs
import warnings
warnings.filterwarnings('ignore')

In [88]:
# df = pd.read_csv("mega_dateclose_final.csv", index_col=0)
# df.index = pd.to_datetime(df.index)
# df.columns = df.columns.str.upper()
# df
# # df.to_csv("mega_dateclose_final.csv")

In [91]:
df = pd.read_csv("merged_trial.csv")
column_list = df['Stock Code'].tolist()
print(column_list)
column_list.append('Date')
df = pd.read_csv("mega_dateclose_final.csv")
filtered_df = df.loc[:, df.columns.intersection(column_list)]
# filtered_df.head()
# filtered_df.to_csv("filtered_df.csv", index=False)


['63MOONS', 'ABAN', 'ADANIENT', 'ALLCARGO', 'ALOKINDS', 'ANANTRAJ', 'ARSHIYA', 'ASHOKLEY', 'ASTAR', 'BAJAJHIND', 'BGRENERGY', 'BI', 'CENTRUM', 'CIEINDIA', 'DISHTV', 'EDELWEISS', 'EDUCOMP', 'EMAMILTD', 'EKC', 'FSL', 'GVT&D', 'GLENMARK', 'GMRAIRPORT', 'GTLINFRA', 'GVKPIL', 'HAVELLS', 'HCLTECH', 'HINDALCO', 'HINDNATGLS', 'HLVLTD', 'HUBTOWN', 'IIFL', 'INDIANB', 'IOB', 'NAUKRI', 'JINDRILL', 'JINDALSAW', 'JMFINANCIL', 'JSWSTEEL', 'JUBLPHARMA', 'KPIL', 'LUPIN', 'MAHLIFE', 'NCC', 'PARSVNATH', 'PATELENG', 'PENINLAND', 'PEL', 'PRAJIND', 'PTC', 'RAIN', 'RELINFRA', 'RELIGARE', 'ROLTA', 'MOTHERSON', 'SEPC', 'RENUKA', 'SIMPLEXINF', 'SUZLON', 'TATAMOTORS', 'TATAPOWER', 'TATASTEEL', 'RAMCOCEM', 'TIMETECHNO', 'TRIVENI', 'UNITECH', 'UNITDSPR', 'IDEA', 'WELCORP', 'WOCKPHARMA']


In [92]:
class long_only_momentum_strategy:
    
    def __init__(self, data, t_1, t_7, t_13, mrr, sigma):
        self.data = data.sort_index() # Sort data alphabetically by stock name (index)
        self.t_1 = t_1                # past 1 month 
        self.t_7 = t_7                # past 7 months
        self.t_13 = t_13              # past 13 months   
        self.mrr = mrr                # 3m MIBOR 
        self.sigma = sigma            # volatility (sigma) over the past 3 years
       
    
    @property
    def price_momentum(self):
        '''Calculating 6 & 12 month Price Momentum'''
        # Ensure the momentum is sorted alphabetically by stock name
        momentum_6 = ((self.data.iloc[self.t_1] / self.data.iloc[self.t_7]) - 1) - self.mrr
        momentum_12 = ((self.data.iloc[self.t_1] / self.data.iloc[self.t_13]) - 1) - self.mrr
        
        return momentum_6.sort_index(), momentum_12.sort_index()
    
    @property
    def risk_adjusted_momentum(self):
        '''Calculating Risk-Adjusted Momentum'''
        momentum_6, momentum_12 = self.price_momentum  # Access price momentum values
        risk_adj_momentum_6 = (momentum_6 / self.sigma).sort_index()
        risk_adj_momentum_12 = (momentum_12 / self.sigma).sort_index()
        
        return risk_adj_momentum_6, risk_adj_momentum_12
    
    @property
    def z_score(self):
        '''Calculating Winsorized Z-Scores'''
        risk_adj_momentum_6, risk_adj_momentum_12 = self.risk_adjusted_momentum
        
        # Calculate Z-scores for 6-month momentum
        momentum_6_sigma = risk_adj_momentum_6.std()
        momentum_6_zscore = (risk_adj_momentum_6 - risk_adj_momentum_6.mean()) / momentum_6_sigma
        winsorized_6_zscore = np.clip(momentum_6_zscore, -3, 3)
        
        # Calculate Z-scores for 12-month momentum
        momentum_12_sigma = risk_adj_momentum_12.std()
        momentum_12_zscore = (risk_adj_momentum_12 - risk_adj_momentum_12.mean()) / momentum_12_sigma
        winsorized_12_zscore = np.clip(momentum_12_zscore, -3, 3)
        
        # Ensure Z-scores are sorted alphabetically
        return winsorized_6_zscore.sort_index(), winsorized_12_zscore.sort_index()
    
    @property
    def combined_score(self):
        '''Calculating Combined Momentum Score'''
        winsorized_6_zscore, winsorized_12_zscore = self.z_score
        
        # Combine the Z-scores with equal weights
        momentum_combined_score = (0.5 * winsorized_6_zscore + 0.5 * winsorized_12_zscore).sort_index()
        
        return momentum_combined_score
    
    @property
    def top_stocks(self):
        '''Pick Top 12 Stocks with Highest Combined Momentum Score'''
        top_stocks = self.combined_score.sort_values(ascending = False).head(10)
        
        return top_stocks

In [93]:
# Calculating past 3years Sigma 


import pandas as pd
import numpy as np

df = pd.read_csv("filtered_df.csv")

date_col = 'Date'

numeric_df = df.drop(columns=[date_col]).apply(pd.to_numeric, errors='coerce')

sigma0 = numeric_df[2773:3517].pct_change().std() * np.sqrt(250)

print(sigma0)




# sigma0 = df[0:3517].pct_change().std() * np.sqrt(250)

63MOONS       0.554145
ABAN          0.645425
ADANIENT      0.572134
ALLCARGO      0.459422
ALOKINDS           NaN
                ...   
TRIVENI       0.760488
UNITDSPR      0.632541
UNITECH       1.013046
WELCORP            NaN
WOCKPHARMA    0.415714
Length: 68, dtype: float64


In [95]:
mrr = 0.045 #https://www.fimmda.org/NSE.aspx
t_1 = 3496
t_7 = 3378
t_13 = 3245

In [119]:
df = pd.read_csv("filtered_df.csv")

date_col = 'Date'

numeric_df = df.drop(columns=[date_col]).apply(pd.to_numeric, errors='coerce')

top_stocks = long_only_momentum_strategy(numeric_df[0:3517], t_1, t_7, t_13, mrr, sigma0)

top_stocks = top_stocks.top_stocks
top_stocks

z_mom = long_only_momentum_strategy(numeric_df[0:3517], t_1, t_7, t_13, mrr, sigma0)
z_mom = z_mom.combined_score
x=z_mom.reset_index()
x.head()
x.columns=["STOCKNAME","Z_MOM"]
x.head()
x.to_csv("z_mom_0609.csv",index=False)

In [97]:
def returns(df, wts):
    ret = np.dot(df, wts)
    
    return ret

In [98]:
tickers = top_stocks.index.to_list()

In [11]:
#df_jul = yf.download(tickers, start='2024-06-28', end='2024-08-01')['Adj Close']
#df_jul

In [12]:
#df_jul.to_csv('df_jul.csv', index=True)

In [103]:
# df_jul = pd.read_csv('df_jul.csv', index_col=0).pct_change().dropna()

In [99]:
wts = top_stocks/top_stocks.sum() # Score weighted 
wts

LUPIN         0.162043
ALLCARGO      0.155472
RENUKA        0.129818
GMRAIRPORT    0.107752
RELIGARE      0.094854
EMAMILTD      0.077100
GTLINFRA      0.072184
PTC           0.070826
MOTHERSON     0.065891
DISHTV        0.064059
dtype: float64

In [16]:
jul_ret = pd.DataFrame({
    'Strategy': returns(df_jul, wts)
}, index=df_jul.index)

# 1st Rebalancing 

In [18]:
#df1 = yf.download(tickers, start='2021-06-30', end = '2024-07-02')['Adj Close']
#df1

In [19]:
#df1.to_csv('1st_reblancing.csv', index=True)

In [20]:
df1 = pd.read_csv('1st_reblancing.csv', index_col=0)
df1

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BEL.NS,BHARTIARTL.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,TRENT.NS,ULTRACEMCO.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2021-06-30,1503.798584,691.226013,3580.988281,2914.788086,745.053711,3706.591309,1208.775513,5929.268555,56.223331,508.497070,...,653.112366,735.329590,337.503357,49.226288,3136.011475,972.911682,1705.278198,846.078369,6634.757324,537.791016
2021-07-01,1487.528320,690.636658,3638.958496,2942.936035,743.211731,3769.983887,1179.567017,5881.269043,58.323814,505.594940,...,661.524658,736.937805,342.124664,49.097588,3132.027832,963.763855,1712.955933,854.092651,6567.440430,531.581726
2021-07-02,1419.453003,697.807312,3693.120850,2926.767822,747.393555,3743.802002,1172.978638,5911.429199,57.060360,507.868256,...,658.720520,744.296997,342.770630,47.935078,3116.984375,967.538635,1727.178711,895.806702,6579.827148,530.842468
2021-07-05,1391.803345,697.463501,3678.183350,2936.313232,757.051453,3767.249023,1190.213379,5985.300293,56.602371,506.852539,...,657.705383,746.928711,343.963226,48.814873,3112.813477,954.438416,1722.405029,897.947205,6579.777832,528.624939
2021-07-06,1412.216187,700.066528,3666.460449,2924.333252,755.060120,3761.779297,1206.953613,6113.133789,57.186707,508.932404,...,650.985107,743.322205,314.943481,49.253712,3057.793213,932.146240,1735.102295,897.847595,6788.533691,524.928955
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-06-25,3171.100098,1456.150024,6285.688477,2858.449951,1270.469482,9659.950195,1603.349976,7074.450195,308.863403,1407.108398,...,1500.441895,1094.599976,955.000000,175.679993,3820.235596,1389.300781,3391.353027,5338.799805,10782.258789,496.850006
2024-06-26,3170.500000,1467.800049,6127.381836,2863.350098,1284.408813,9474.650391,1600.250000,7158.200195,306.021088,1451.411621,...,1516.341553,1086.900024,951.849976,172.559998,3837.553223,1374.996704,3361.750000,5363.799805,11077.407227,495.200012
2024-06-27,3175.149902,1485.500000,6182.996582,2880.850098,1287.955933,9417.450195,1603.099976,7166.750000,303.677429,1467.621338,...,1511.457031,1085.599976,972.099976,174.160004,3915.481445,1393.679688,3380.600098,5329.000000,11647.626953,510.799988


In [21]:
sigma1 = df1.pct_change().dropna().std() * np.sqrt(250)

In [22]:
t_1 = 720
t_7 = 600
t_13 = 476

In [23]:
top_stocks = long_only_momentum_strategy(df, t_1, t_7, t_13, mrr, sigma0)

top_stocks = top_stocks.top_stocks
top_stocks

TRENT.NS         2.748404
BAJAJ-AUTO.NS    1.953101
BEL.NS           1.730616
NTPC.NS          1.701589
BPCL.NS          1.659711
COALINDIA.NS     1.037085
BHARTIARTL.NS    0.969310
HEROMOTOCO.NS    0.962237
TATAMOTORS.NS    0.951302
POWERGRID.NS     0.933038
ONGC.NS          0.765823
SUNPHARMA.NS     0.739620
dtype: float64

In [24]:
#tickers = top_stocks.index.to_list()

In [25]:
#df_aug = yf.download(tickers, start = '2024-07-31', end = '2024-09-01')['Adj Close']
#df_aug

In [26]:
#df_aug.to_csv('df_aug.csv', index=True)

In [27]:
df_aug = pd.read_csv('df_aug.csv', index_col=0).pct_change().dropna()

In [28]:
wts = top_stocks/top_stocks.sum()
wts

TRENT.NS         0.170160
BAJAJ-AUTO.NS    0.120921
BEL.NS           0.107147
NTPC.NS          0.105350
BPCL.NS          0.102757
COALINDIA.NS     0.064209
BHARTIARTL.NS    0.060012
HEROMOTOCO.NS    0.059574
TATAMOTORS.NS    0.058897
POWERGRID.NS     0.057767
ONGC.NS          0.047414
SUNPHARMA.NS     0.045792
dtype: float64

In [29]:
aug_ret = pd.DataFrame({
    'Strategy': returns(df_aug, wts)
}, index=df_aug.index)

# 2nd Rebalancing 

In [31]:
#df2 = yf.download(tickers, start='2021-08-30', end = '2024-09-01')['Adj Close']
#df2

In [32]:
#df2.to_csv('2nd_rebalancing.csv', index=True)

In [33]:
df2 = pd.read_csv('2nd_rebalancing.csv', index_col=0)
df2

,ADANIENT.NS,ADANIPORTS.NS,APOLLOHOSP.NS,ASIANPAINT.NS,AXISBANK.NS,BAJAJ-AUTO.NS,BAJAJFINSV.NS,BAJFINANCE.NS,BEL.NS,BHARTIARTL.NS,...,SUNPHARMA.NS,TATACONSUM.NS,TATAMOTORS.NS,TATASTEEL.NS,TCS.NS,TECHM.NS,TITAN.NS,TRENT.NS,ULTRACEMCO.NS,WIPRO.NS
Date,,,,,,,,,,,,,,,,,,,,,
2021-08-30,1513.780396,717.059875,4709.344238,3027.816895,780.648621,3451.237549,1653.528442,7073.784180,58.134289,600.203674,...,763.172058,838.747437,290.147552,60.788090,3476.844238,1277.125244,1853.370361,987.543701,7580.027832,623.340820
2021-08-31,1584.700439,735.035461,4919.196777,3118.006104,783.087952,3458.148682,1712.274414,7428.188965,58.955532,642.380798,...,769.812988,843.085022,285.526245,61.195290,3556.830322,1302.727173,1895.944336,1003.084656,7706.611328,631.718384
2021-09-01,1562.341431,731.548401,4987.992676,3216.766113,795.434265,3486.395752,1676.084351,7423.352051,59.966290,644.654175,...,764.432312,845.521790,293.427155,59.547516,3489.666504,1292.513428,1913.506714,990.632019,7642.163574,632.802551
2021-09-02,1567.681641,740.487061,4992.398438,3215.646484,799.217712,3451.747803,1667.791870,7402.473145,60.061043,644.750916,...,765.838135,849.420715,291.340118,60.134052,3604.079834,1293.863281,1940.836914,996.708862,7824.978027,642.067200
2021-09-03,1562.690674,741.518494,4963.741211,3251.877686,794.538147,3485.653564,1670.897217,7427.103027,62.730068,636.963562,...,765.304871,847.861023,293.725281,60.916794,3609.058594,1297.597656,1992.340088,991.927124,7802.445801,645.664551
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-08-26,3069.000000,1482.550049,6780.000000,3171.350098,1170.300049,10432.549805,1686.199951,6778.350098,306.700012,1513.550049,...,1772.449951,1220.050049,1092.400024,155.699997,4491.492676,1625.548950,3630.199951,6925.600098,11337.099609,520.000000
2024-08-27,3067.100098,1478.349976,6781.450195,3166.199951,1181.250000,10501.599609,1719.000000,6863.600098,300.899994,1522.750000,...,1789.400024,1209.599976,1077.250000,154.699997,4486.205566,1610.137329,3551.250000,6869.899902,11300.349609,517.150024
2024-08-28,3028.000000,1469.800049,6776.450195,3125.500000,1170.949951,10656.750000,1713.500000,6900.000000,299.950012,1556.349976,...,1811.849976,1201.150024,1074.550049,153.699997,4495.083496,1612.862793,3527.500000,7242.100098,11234.799805,534.599976


In [34]:
sigma2 = df2.pct_change().dropna().std() * np.sqrt(200)

In [35]:
t_1 = 720
t_7 = 600
t_13 = 476

In [36]:
top_stocks = long_only_momentum_strategy(df, t_1, t_7, t_13, mrr, sigma0)

top_stocks = top_stocks.top_stocks
top_stocks

TRENT.NS         2.748404
BAJAJ-AUTO.NS    1.953101
BEL.NS           1.730616
NTPC.NS          1.701589
BPCL.NS          1.659711
COALINDIA.NS     1.037085
BHARTIARTL.NS    0.969310
HEROMOTOCO.NS    0.962237
TATAMOTORS.NS    0.951302
POWERGRID.NS     0.933038
ONGC.NS          0.765823
SUNPHARMA.NS     0.739620
dtype: float64

In [37]:
tickers = top_stocks.index.to_list()
tickers

['TRENT.NS',
 'BAJAJ-AUTO.NS',
 'BEL.NS',
 'NTPC.NS',
 'BPCL.NS',
 'COALINDIA.NS',
 'BHARTIARTL.NS',
 'HEROMOTOCO.NS',
 'TATAMOTORS.NS',
 'POWERGRID.NS',
 'ONGC.NS',
 'SUNPHARMA.NS']

In [38]:
#df_sept = yf.download(tickers, start='2024-08-30', end = '2024-10-01')['Adj Close']
#df_sept

In [39]:
#df_sept.to_csv('df_sept.csv', index=True)

In [40]:
df_sept = pd.read_csv('df_sept.csv', index_col=0).pct_change().dropna()
df_sept

,BAJAJ-AUTO.NS,BEL.NS,BHARTIARTL.NS,BPCL.NS,COALINDIA.NS,HEROMOTOCO.NS,NTPC.NS,ONGC.NS,POWERGRID.NS,SUNPHARMA.NS,TATAMOTORS.NS,TRENT.NS
Date,,,,,,,,,,,,
2024-09-02,0.021535,-0.008019,-0.011139,0.002237,-0.007810,0.022510,-0.014897,-0.013757,-0.005483,-0.003129,-0.016826,-0.001474
2024-09-03,-0.007410,0.000842,-0.006841,-0.008509,-0.002400,0.012244,-0.008780,-0.012262,-0.002682,-0.002450,-0.006910,-0.014745
2024-09-04,-0.007239,0.006058,0.000833,0.005205,-0.030697,0.006597,-0.003199,-0.024209,-0.005229,0.011786,-0.004285,0.013787
2024-09-05,-0.009846,-0.027931,-0.009412,0.009657,-0.013204,0.008876,-0.004567,-0.009542,-0.004956,-0.003465,-0.010459,0.003887
2024-09-06,-0.002363,-0.024088,-0.005138,-0.023704,-0.017002,0.001665,-0.020955,-0.008349,-0.004377,-0.001068,-0.018519,-0.008943
2024-09-09,0.001616,-0.007229,0.002209,-0.012353,-0.008392,0.000270,-0.012538,-0.032060,-0.003790,-0.001480,-0.010149,0.004660
2024-09-10,0.012920,0.014917,0.022818,-0.005319,0.013727,-0.013159,0.016545,-0.011040,0.017045,0.007959,-0.002792,0.000182
2024-09-11,0.039408,0.008049,-0.000032,-0.016476,-0.014559,-0.002690,-0.016780,-0.034844,-0.002843,0.005092,-0.057444,0.001527
2024-09-12,0.026509,0.012671,0.043700,0.011903,0.024176,0.026298,0.039009,0.030669,0.015006,0.010673,0.010089,0.008624


In [41]:
sept_ret = pd.DataFrame({
    'Strategy': returns(df_sept, wts)
}, index=df_sept.index)

In [42]:
# 3 months Returns of Strategy 
df_returns = pd.concat([jul_ret, aug_ret, sept_ret])
df_returns

,Strategy
Date,
2024-07-01,0.001931
2024-07-02,-0.005543
2024-07-03,0.006562
2024-07-04,0.004622
2024-07-05,0.012029
...,...
2024-09-24,0.007298
2024-09-25,0.001443
2024-09-26,0.008582


In [43]:
#nifty50 = yf.download('^NSEI', start='2024-06-28', end= '2024-10-01')['Adj Close']
#nifty50

In [44]:
#nifty50.to_csv('nifty50.csv', index=True)

In [45]:
nse = pd.read_csv('nifty50.csv', index_col=0).pct_change().dropna()
nse

,Adj Close
Date,
2024-07-01,0.005470
2024-07-02,-0.000750
2024-07-03,0.006742
2024-07-04,0.000644
2024-07-05,0.000893
...,...
2024-09-24,0.000052
2024-09-25,0.002458
2024-09-26,0.008149


In [46]:
df_backtest = pd.DataFrame({
    'Strategy': df_returns['Strategy'],
    'Nifty 50': nse['Adj Close']
}, index=df_returns.index)
df_backtest

,Strategy,Nifty 50
Date,,
2024-07-01,0.001931,0.005470
2024-07-02,-0.005543,-0.000750
2024-07-03,0.006562,0.006742
2024-07-04,0.004622,0.000644
2024-07-05,0.012029,0.000893
...,...,...
2024-09-24,0.007298,0.000052
2024-09-25,0.001443,0.002458
2024-09-26,0.008582,0.008149


In [47]:
df_backtest.index = pd.to_datetime(df_backtest.index)

In [48]:
#qs.reports.html(df_backtest['Strategy'], benchmark=df_backtest['Nifty 50'], rf=mrr)

10.0